In [ ]:
import os
import re
import time
import sys
import papermill as pm
import numpy as np
import pandas as pd
import holoviews as hv
from papermill import PapermillExecutionError

In [ ]:
%load_ext autoreload
%autoreload 2
minian_path = "../"
sys.path.append(minian_path)
from minian.utilities import open_minian_mf, load_videos
from minian.visualization import CNMFViewer, VArrayViewer
hv.notebook_extension('bokeh')

In [ ]:
notebook_path = '../pipeline.ipynb'
dpath = '../demo_movies'

In [ ]:
params = {
    'minian_path' : "/absolute/path/to/minian",
    'dpath' : ".",
    'chunks' : {"frame": 1000, "height": 50, "width": 50, "unit_id": 100},
    'subset' : None,
    'subset_mc': None,
    'in_memory' : False,
    'interactive' : False,
    'output_size' : 100,
    'param_load_videos' : {
        'pattern': 'msCam[0-9]+\.avi$',
        'dtype': 'f4',
        'in_memory': False,
        'downsample': {'frame': 2},
        'downsample_strategy': 'subset'},
    'param_glow_removal' : {
        'method': 'uniform',
        'wnd': 51},
    'param_brightspot_removal' : {
        'thres': 2},
    'param_first_denoise' : {
        'method': 'gaussian',
        'sigmaX': 0,
        'ksize': (5, 5)},
    'param_second_denoise' : {
        'method': 'median',
        'ksize': 5},
    'param_estimate_shift' : {
        'dim': 'frame',
        'on': 'first',
        'pad_f': 1,
        'pct_thres': 99.99},
    'param_background_removal' : {
        'method': 'tophat',
        'wnd': 10},
    'param_seeds_init' : {
        'wnd_size': 2000,
        'method': 'rolling',
        'stp_size': 1000,
        'nchunk': 100,
        'max_wnd': 10},
    'param_gmm_refine' : {
        'q': (0.1, 99.9),
        'n_components': 2,
        'valid_components': 1},
    'param_pnr_refine' : {
        'noise_freq': 0.06,
        'thres': 1,
        'q': (0.1, 99.9)},
    'param_ks_refine' : {
        'sig': 0.05},
    'param_seeds_merge' : {
        'thres_dist': 5,
        'thres_corr': 0.8,
        'noise_freq': 0.06},
    'param_initialize' : {
        'thres_corr': 0.8,
        'wnd': 10},
    'param_get_noise' : {
        'noise_range': (0.06, 0.5),
        'noise_method': 'logmexp'},
    'param_first_spatial' : {
        'dl_wnd': 5,
        'sparse_penal': 0.1,
        'update_background': False,
        'post_scal': True,
        'zero_thres': 'eps'},
    'param_first_temporal' : {
        'noise_freq': 0.06,
        'sparse_penal': 1,
        'p': 2,
        'add_lag': 20,
        'use_spatial': False,
        'chk': {"frame": 1000, "height": 50, "width": 50, "unit_id": 100},
        'jac_thres': 0.2,
        'zero_thres': 1e-8,
        'max_iters': 200,
        'use_smooth': True,
        'scs_fallback': False,
        'post_scal': True},
    'param_first_merge' : {
        'thres_corr': 0.9},
    'param_second_spatial' : {
        'dl_wnd': 5,
        'sparse_penal': 0.05,
        'update_background': False,
        'post_scal': True,
        'zero_thres': 'eps'},
    'param_second_temporal' : {
        'noise_freq': 0.06,
        'sparse_penal': 1,
        'p': 2,
        'add_lag': 20,
        'use_spatial': False,
        'chk': {"frame": 1000, "height": 50, "width": 50, "unit_id": 100},
        'jac_thres': 0.1,
        'zero_thres': 1e-8,
        'max_iters': 500,
        'use_smooth': True,
        'scs_fallback': False,
        'post_scal': True},
    'param_second_merge' : {
        'thres_corr': 0.9},
    'param_save_minian' : {
        'dpath': ".",
        'fname': 'minian',
        'backend': 'zarr',
        'meta_dict': {"session_id": -1, "session": -2, "animal": -3}}
}
params_override = dict()

In [ ]:
vlist = []
for cur_fd, dirs, files in os.walk(dpath):
    try:
        cur_vid = load_videos(cur_fd, **params['param_load_videos'])
        vlist.append(cur_vid)
    except FileNotFoundError:
        pass

In [ ]:
vaviewer = VArrayViewer(vlist, summary=['diff'], layout=False, histogram=False)

In [ ]:
hv.output(size=60)
vaviewer.show()

In [ ]:
params_override.update({fn[0]: dict(subset_mc = str(box)) for fn, box in vaviewer.mask.items()})

In [ ]:
summary_list = []
err_dict = dict()
for root, dirs, files in os.walk(dpath):
    pattern = params['param_load_videos']['pattern']
    vfiles = list(filter(lambda f: re.search(pattern, f), files))
    if vfiles:
        ts = int(time.time())
        params['dpath'] = root
        params['param_save_minian']['dpath'] = root
        params['param_save_minian']['fname'] = "minian.{}".format(ts)
        vname = os.path.split(root)[-1]
        try:
            params.update(params_override[vname])
            print("using parameters override")
        except KeyError:
            print("using default parameters")
        out_nb_path = os.path.join(root, 'minian_result.{}.ipynb'.format(ts))
        try:
            exe_result = pm.execute_notebook(
                notebook_path,
                out_nb_path,
                parameters = params)
        except PapermillExecutionError as e:
            print(e)
            err_dict[root] = e
            continue
        res_df = pm.read_notebook(out_nb_path).dataframe
        df_list = []
        for ir, row in res_df.iterrows():
            if row['name'].startswith('param_'):
                cur_rlist = [dict(
                    function=row['name'][6:],
                    name=k, value=v,
                    type=row['type'],
                    filename=row['filename']
                ) for k, v in row['value'].items()]
                df_list.append(pd.DataFrame(cur_rlist))
            else:
                df_list.append(row.to_frame().T)
        res_df = pd.concat(df_list, ignore_index=True, sort=False)
        res_df['path'] = root
        summary_list.append(res_df)
    else:
        # print("no data found under {}".format(root))
        pass
summary = pd.concat(summary_list, ignore_index=True, sort=False)

In [ ]:
summary.to_pickle(os.path.join(dpath, "summary.pkl"))

In [ ]:
%%time
%%capture
minian = open_minian_mf(
    dpath, ['animal', 'session'], backend='zarr',
    chunks=dict(height='auto', width='auto', frame='auto', unit_id='auto'))

In [ ]:
vaviewer = VArrayViewer(
    minian['org'], meta_dims=['animal', 'session'],
    summary=minian['shifts'].rename(variable='sum_var'), layout=False)

In [ ]:
hv.output(size=60)
vaviewer.show()

In [ ]:
cnmfviewer = CNMFViewer(minian)

In [ ]:
hv.output(size=60)
cnmfviewer.show()